In [1]:
from google.colab import drive


FOLDERNAME = 'damaged_structures_detector/carr'

drive.mount('/content/drive', force_remount=True)

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My\ Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler, RandomSampler, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from PIL import Image, ImageOps
import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np
from sklearn.metrics import confusion_matrix
import time

import seaborn as sns
from __future__ import print_function 
from __future__ import division
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.5.0+cu101
Torchvision Version:  0.6.0+cu101


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


print('using device:', device)

using device: cpu


In [0]:
# BATCH_SIZE = 128

# first_transform = transforms.Compose([
#         transforms.Resize(224),
#         transforms.ToTensor() ])

# train_set = datasets.ImageFolder(FOLDERNAME, transform = first_transform)
# train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, num_workers = 16)

In [0]:
# i = 1
# num_batches = int(np.ceil(len(train_set.imgs) / BATCH_SIZE))
# mean = 0.
# std = 0.
# nb_samples = 0.
# for data, labels in train_loader:
#     batch_samples = data.size(0)
#     data = data.view(batch_samples, data.size(1), -1)
#     mean += data.mean(2).sum(0)
#     std += data.std(2).sum(0)
#     nb_samples += batch_samples
#     print('iteration: ', i, ' of ',  num_batches)
#     i += 1

# mean /= nb_samples
# std /= nb_samples

In [0]:
BATCH_SIZE = 128
FOLDERNAME = 'damaged_structures_detector/images/test'

data_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.3662, 0.3452, 0.3384],
                             std=[0.1552, 0.1500, 0.1475])
    ])


test_dataset = datasets.ImageFolder(FOLDERNAME, transform = data_transform)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = 16)

In [0]:
%%capture
MODEL_PATH = "damaged_structures_detector/checkpoints/best_model.pth"
model = torch.load(MODEL_PATH, map_location=device)
model.to(device)
model.eval()

In [0]:
running_corrects = 0

for inputs, labels in test_loader:
  inputs = inputs.to(device)
  labels = labels.to(device)

  with torch.no_grad():
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)

  running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / len(test_loader.dataset)

In [11]:
## after normalizing with own mean and std acc = 0.8367
## after normalizing with training set mean and std acc = 0.9736
test_acc

tensor(0.9367, dtype=torch.float64)